# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Признаки:
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Нам нужно:
1. Загрузить данные
2. Провести исследовательский анализ данных
3. Обучить разные модели 
4. Сравнить результаты и выбрать лучшую

Для оценки качества моделей используем метрику RMSE.

## Подготовка данных

Импортируем необходимые библиотеки и откроем файл

In [223]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb

In [224]:
df = pd.read_csv('/datasets/autos.csv')

Посмотрим на общую информацию

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [226]:
df.isna().mean()

DateCrawled          0.000000
Price                0.000000
VehicleType          0.105794
RegistrationYear     0.000000
Gearbox              0.055967
Power                0.000000
Model                0.055606
Kilometer            0.000000
RegistrationMonth    0.000000
FuelType             0.092827
Brand                0.000000
Repaired             0.200791
DateCreated          0.000000
NumberOfPictures     0.000000
PostalCode           0.000000
LastSeen             0.000000
dtype: float64

In [227]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Есть пропущенные значения в столбцах VehicleType, Gearbox, Model, FuelType, NotRepaired. По всем столбцам укажем no info т.к сложно определить по какой причине отсутсвует информация, в столбце была машина в ремонте или нет информация пропущена скорее всего по причине того что ремонта не было, предположим так и укажем Нет вместо пропущенных значений

In [228]:
df.columns = df.columns.str.lower()

In [229]:
df['repaired'] = df['repaired'].fillna('no')
df['vehicletype'] = df['vehicletype'].fillna('no info')
df['gearbox'] = df['gearbox'].fillna('no info')
df['model'] = df['model'].fillna('no info')
df['fueltype'] = df['fueltype'].fillna('no info')

Готово. Приведем столбец notrepaired в числовой вид

In [230]:
df['repaired'] = df['repaired'].map({'yes': 1, 'no': 0})

Удалим лишние столбцы, например столбец numberofpictures, во всех строках указано 0, и столбцы с датами которые нам не понадобятся тоже удалим

In [231]:
df['numberofpictures'].sum()

0

In [232]:
df = df.drop(labels=['numberofpictures','datecrawled','datecreated','lastseen'],axis=1)

Прверим данные на дубликаты и удалим их

In [233]:
df.duplicated().sum()

23697

In [234]:
df = df.drop_duplicates().reset_index(drop=True)

In [235]:
df.duplicated().sum()

0

Посмотрим что получилось

In [236]:
df.head()

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired,postalcode
0,480,no info,1993,manual,0,golf,150000,0,petrol,volkswagen,0,70435
1,18300,coupe,2011,manual,190,no info,125000,5,gasoline,audi,1,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,0,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,0,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,0,60437


In [60]:
df.isna().sum().sum()

0

Пропущенных значений нет, данные готовы

In [237]:
df.describe()

,price,registrationyear,power,kilometer,registrationmonth,repaired,postalcode
count,330672.000000,330672.000000,330672.000000,330672.000000,330672.000000,330672.000000,330672.000000
mean,4376.277744,2004.204420,109.705082,128276.963275,5.689396,0.104778,50683.262577
std,4505.024019,90.592673,194.925268,37934.650280,3.729673,0.306267,25805.591756
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.000000,1067.000000
25%,1000.000000,1999.000000,68.000000,125000.000000,3.000000,0.000000,30179.000000
50%,2690.000000,2003.000000,103.000000,150000.000000,6.000000,0.000000,49477.000000
75%,6299.000000,2008.000000,140.000000,150000.000000,9.000000,0.000000,71334.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,1.000000,99998.000000


Необходимо удалить аномальные значения, в столбце Price удалим стоимость 0

In [238]:
df[df['price'] == 0]['price'].count()

9983

In [239]:
df = df[df['price'] != 0]

В столбце Год регистрации есть даты больше 2017. Максимальный год — дата скачивания анкеты 2016.

In [240]:
df[df['registrationyear'] >= 2017]['registrationyear'].count()

13167

In [241]:
df[df['registrationyear'] < 1900]['registrationyear'].count()

50

In [242]:
df = df[(df['registrationyear'] < 2017) & (df['registrationyear'] > 1900)]

Мощность возьмем от 50 до 500

In [243]:
df[df['power'] >= 500]['power'].count()

417

In [244]:
df[df['power'] <= 50]['power'].count()

40802

In [245]:
df = df[(df['power'] < 500) & (df['power'] > 50)]

Месяц регистрации 0 явно какая то ошибка, удалим их

In [246]:
df[df['registrationmonth'] == 0]['registrationmonth'].count()

16254

In [247]:
df = df[df['registrationmonth'] != 0]

Здесь я поменяла тип категориальных данных на Category для построения модели LightGBM

In [248]:
df[['vehicletype','gearbox','model','fueltype','brand']] = df[['vehicletype','gearbox','model','fueltype','brand']].astype('category')

In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249999 entries, 1 to 330671
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   price              249999 non-null  int64   
 1   vehicletype        249999 non-null  category
 2   registrationyear   249999 non-null  int64   
 3   gearbox            249999 non-null  category
 4   power              249999 non-null  int64   
 5   model              249999 non-null  category
 6   kilometer          249999 non-null  int64   
 7   registrationmonth  249999 non-null  int64   
 8   fueltype           249999 non-null  category
 9   brand              249999 non-null  category
 10  repaired           249999 non-null  int64   
 11  postalcode         249999 non-null  int64   
dtypes: category(5), int64(7)
memory usage: 16.7 MB


Готово

## Обучение моделей

Проведем кодирование данных

In [250]:
#df = pd.get_dummies(df, drop_first=True)
df.head()

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired,postalcode
1,18300,coupe,2011,manual,190,no info,125000,5,gasoline,audi,1,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,0,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,0,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,0,60437
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,1,33775


Разделим данные на обучающую, валидационную и тестовую выборки

In [251]:
target = df['price']
features = df.drop('price', axis=1)

In [252]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, train_size=0.6, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

Закодируем выборки двумя способами для разных моделей

In [255]:
features_train_ohe = pd.get_dummies(features_train, drop_first=True)
features_valid_ohe = pd.get_dummies(features_valid, drop_first=True)
features_test_ohe = pd.get_dummies(features_test, drop_first=True)

In [256]:
numeric = ['registrationyear', 'power', 'kilometer', 'registrationmonth', 'postalcode']
categorical = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand']

In [257]:
import copy
features_train_enc = copy.deepcopy(features_train)
features_valid_enc = copy.deepcopy(features_valid)

In [258]:
enc = OrdinalEncoder()
enc.fit(features_train_enc[categorical])
features_train_enc[categorical] = enc.transform(features_train_enc[categorical])

enc.fit(features_valid_enc[categorical])
features_valid_enc[categorical] = enc.transform(features_valid_enc[categorical])

Отмасштабируем признаки

In [259]:
scaler = StandardScaler()
scaler.fit(features_train_ohe[numeric])
features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_valid_ohe[numeric] = scaler.transform(features_valid_ohe[numeric])

In [260]:
scaler = StandardScaler()
scaler.fit(features_train_enc[numeric])
features_train_enc[numeric] = scaler.transform(features_train_enc[numeric])
features_valid_enc[numeric] = scaler.transform(features_valid_enc[numeric])

Обучим модель линейной регрессии, для оценки качества моделей применим метрику RMSE

In [200]:
model = LinearRegression()
%time
model.fit(features_train_ohe, target_train)
%time
predictions = model.predict(features_valid_ohe)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
CPU times: user 24 µs, sys: 1e+03 ns, total: 25 µs
Wall time: 9.54 µs


In [264]:
rmse_lr = mean_squared_error(target_valid, predictions)**0.5
print(rmse_lr)

2574.370523135408


Для итогового анализа соберем все результаты в словарь

In [265]:
result = {}
result['LinearRegression'] = ['2574', '5.96 s', '9.54']

Посмотрим какой результат покажет дерево

In [208]:
for depth in range(1,6):
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    %time
    model.fit(features_train_enc,target_train)
    %time
    predictions_valid = model.predict(features_valid_enc)
    
    print("max_depth =", depth, ": ", end='')
    print(mean_squared_error(target_valid, predictions_valid)**0.5)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
max_depth = 1 : 3693.5399942618033
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
CPU times: user 0 ns, sys: 24 µs, total: 24 µs
Wall time: 28.1 µs
max_depth = 2 : 3239.559121743341
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs
CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs
max_depth = 3 : 2958.1321977975263
CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.3 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
max_depth = 4 : 2624.7050795973028
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
max_depth = 5 : 2433.6420822443133


In [266]:
result['DecisionTreeRegressor'] = ['2433', '7.63 s', '6.91 s']

Обучим лес и используем гридсерч

In [213]:
param = {'n_estimators':range(10,52,103)}
model = RandomForestRegressor(random_state=12345)
grid = GridSearchCV(model,param,cv=5,scoring='neg_mean_squared_error')
%time
grid.fit(features_train_enc,target_train)
%time
predicted_valid = grid.predict(features_valid_enc)
rmse = (-grid.best_score_) ** 0.5
print('RMSE_rfr =', rmse, 'кол-во деревьев',grid.best_params_)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
RMSE_rfr = 1750.1389209569338 кол-во деревьев {'n_estimators': 10}


In [267]:
result['RandomForestRegressor'] = ['1750', '5.72 s', '5.96 s']

Модель LightGBM

In [216]:
model = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=7, learning_rate=0.1, n_estimators=200)
%time
model.fit(features_train, target_train)
%time
predicted_valid = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predicted_valid)**0.5
print('RMSE_lightgbm =', rmse)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
RMSE_lightgbm = 1649.5489375431312


In [268]:
result['LGBMRegressor'] = ['1649', '5.96 s', '6.44 s']

Модель CatBoost

In [261]:
cat_features = categorical
model = CatBoostRegressor(random_state=12345, iterations=10,verbose=10)
%time
model.fit(features_train, target_train, cat_features) 
%time
pred_valid = model.predict(features_valid) 
rmse = mean_squared_error(target_valid, pred_valid)**0.5
print(rmse)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
Learning rate set to 0.5
0:	learn: 3271.9695532	total: 125ms	remaining: 1.13s
9:	learn: 1905.7717545	total: 641ms	remaining: 0us
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
1894.2358347784495


In [271]:
result['CatBoostRegressor'] = ['1894', '6.2 s', '6.2 s']

Все модели успешно обучены, результаты собраны в один словарь. Далее, проанализируем их и выберем лучшую модель

## Анализ моделей

Сравним результаты и проверим на тестовой выборке лучшую модель

In [272]:
result_fin = pd.DataFrame.from_dict(data=result, columns=['RMSE', 'TimeFit', 'TimePredict'], orient='index')
result_fin

,RMSE,TimeFit,TimePredict
LinearRegression,2574,5.96 s,9.54
DecisionTreeRegressor,2433,7.63 s,6.91 s
RandomForestRegressor,1750,5.72 s,5.96 s
LGBMRegressor,1649,5.96 s,6.44 s
CatBoostRegressor,1894,6.2 s,6.2 s


Модель LGBMRegressor показала лучшие результаты и по времени и по качеству, у модели RandomForestRegressor rmse не сильно хуже, и время предсказания чуть быстрее

In [273]:
%%time
model = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=7, learning_rate=0.1, n_estimators=200)
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
rmse = mean_squared_error(target_test, predicted_test)**0.5
print('RMSE_lightgbm =', rmse)

RMSE_lightgbm = 1602.9451549730547
CPU times: user 9.65 s, sys: 125 ms, total: 9.77 s
Wall time: 9.8 s


In [274]:
dummy_mod = DummyRegressor(strategy='mean')
dummy_mod.fit(features_train,target_train)
pred = dummy_mod.predict(features_test)
rmse = mean_squared_error(target_test, pred)**0.5
print('RMSE_Dummy', rmse)

RMSE_Dummy 4682.372402888791


## Вывод 

Перед нами стояла задача построить модель, которая умеет определять рыночную стоимость автомобилей с пробегом. Для этого мы выполнили следующие действия:
1. Загрузили и изучили данные
2. Заполнили пропущенные значения и подготовили данные
3. Обучили разные модели(Линейную регрессию, дерево, лес, LGBM и CatBoost)
4. Проанализировали результаты и пришли к выводу, что наилучшее качество показали модели LGBM и лес, у LGBM качество чуть лучше, но для заказчика так же важно и время обучения. Лес обучался дольше всех моделей, LGBM обучается намного быстрее. Именну эту модель мы и рекомендуем использовать при разработке приложения